In [11]:
BATCH_SIZE = 48
LR = 1e-5

In [12]:
!pip install datasets evaluate transformers[sentencepiece] accelerate

In [13]:
import datasets

dataset = datasets.load_dataset("csv", data_files="Datasets/train.csv")
dataset_val = datasets.load_dataset("csv", data_files="Datasets/val.csv")
dataset_test = datasets.load_dataset("csv", data_files="Datasets/test.csv")
dataset["val"] = dataset_val["train"]
dataset["test"] = dataset_test["train"]


In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question1', 'question2', 'labels'],
        num_rows: 323431
    })
    val: Dataset({
        features: ['question1', 'question2', 'labels'],
        num_rows: 40428
    })
    test: Dataset({
        features: ['question1', 'question2', 'labels'],
        num_rows: 40428
    })
})

In [15]:
dataset["val"].features

{'question1': Value(dtype='string', id=None),
 'question2': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None)}

In [16]:
from transformers import AutoTokenizer, DataCollatorWithPadding

# change model to finetune here
checkpoint = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(data):
    return tokenizer(data["question1"], data["question2"], truncation = True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer)
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['question1', 'question2', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 323431
    })
    val: Dataset({
        features: ['question1', 'question2', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 40428
    })
    test: Dataset({
        features: ['question1', 'question2', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 40428
    })
})

In [17]:
tokenized_dataset = tokenized_dataset.remove_columns(["question1", "question2"])
tokenized_dataset = tokenized_dataset.with_format("torch")
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 323431
    })
    val: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 40428
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 40428
    })
})

In [18]:
small_train_dataset = tokenized_dataset["train"].select(range(10000))
small_train_dataset
small_val_dataset = tokenized_dataset["val"].select(range(10000))
small_train_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10000
})

In [19]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
import torch


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir=f"hyper-{BATCH_SIZE}-{LR}", 
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=BATCH_SIZE,
                                  per_device_eval_batch_size=BATCH_SIZE,
                                  num_train_epochs = 3,
                                  learning_rate=LR
                                 )

In [21]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")
metric2 = evaluate.load("confusion_matrix")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)#", metric2.compute(predictions=predictions, references=labels)


In [22]:
small_train_dataset = tokenized_dataset["train"].select(range(10000))
small_train_dataset
small_val_dataset = tokenized_dataset["val"].select(range(10000))
small_train_dataset
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset if False else tokenized_dataset["train"],
    eval_dataset=small_val_dataset if False else tokenized_dataset["val"] ,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [23]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,0.320200,0.297891,0.870684
2,0.285300,0.277379,0.881419
3,0.266700,0.274895,0.884288


TrainOutput(global_step=20217, training_loss=0.30541310968647645, metrics={'train_runtime': 1643.8708, 'train_samples_per_second': 590.249, 'train_steps_per_second': 12.298, 'total_flos': 4608189934069956.0, 'train_loss': 0.30541310968647645, 'epoch': 3.0})

In [ ]:
trainer.train(True)

In [ ]:
# resume training from latest checkpoint

trainer.train(True)